In [28]:
import fastf1, datetime
import pandas as pd

fastf1.Cache.enable_cache("cache")
current_datetime = datetime.datetime.now()
points = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

In [29]:
def get_safety_car_laps(race_status):
    count = 0
    for status in race_status:
        if '4' in str(status) or '5' in str(status) or '6' in str(status) or '7' in str(status):
            count += 1
    return count

In [30]:
# session = fastf1.get_session(2018, 'Bahrain', 'Race')
# session.load()

# laps = session.laps
# race_status = laps.groupby('LapNumber')['TrackStatus'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# print(race_status)
# print(get_safety_car_laps(race_status))

In [ ]:
from track_data import track_data, alternate_names

gp_df = pd.DataFrame(columns=['grand_prix', 'laps', 'length_km', 'turn_count', 'drs_zones', 'pit_lane_length_m', 'degradation_level', 'street_circuit', 'avg_track_temp', 'avg_air_temp', 'avg_humidity', 'rainfall_mm', 'safety_car_laps', 'best_strategy'])

for year in range(2018, current_datetime.year + 1):
    schedule = fastf1.get_event_schedule(year)
    for _, event in schedule.iterrows():
        # If event format is not pre-season testing
        if event['EventFormat'] == 'testing':
            continue
        try:
            session = fastf1.get_session(year, event['EventName'], 'Race')
            session.load(weather=True)
            print(f"Loaded session for {event['EventName']} {year}")

            session_date = session.date.to_pydatetime()
            if session_date > current_datetime:
                break

            weather = session.weather_data
            avg_track_temp = weather['TrackTemp'].mean()
            avg_air_temp = weather['AirTemp'].mean()
            avg_humidity = weather['Humidity'].mean()
            rain = weather['Rainfall'].mean()

            laps = session.laps
            race_status = laps.groupby('LapNumber')['TrackStatus'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
            safety_car_laps = get_safety_car_laps(race_status)

            track_info = track_data.get(event['Location'])
            if track_info is None:
                for track_list in alternate_names:
                    if event['Location'] in track_list:
                        track_info = track_data.get(track_list[0])
                        break

            stint_data = (
                laps.groupby(["Driver", "Stint", "Compound"])
                .agg(stint_length=("LapNumber", "count"))
                .reset_index()
            )

            statuses = session.results['Status'].fillna('').astype(str)
            dnf_mask = (statuses != 'Finished') & (statuses != 'Lapped') & (~statuses.str.startswith('+'))
            dnf_drivers = session.results[dnf_mask]['Abbreviation'].tolist()

            strategies = pd.DataFrame(columns=['Driver', 'Strategy'])

            for driver in stint_data['Driver'].unique():
                if driver not in dnf_drivers:
                    driver_stints = stint_data[stint_data['Driver'] == driver]
                    strategy_compounds = tuple(driver_stints['Compound'].tolist())
                    strategy_lengths = tuple(driver_stints['stint_length'].tolist())
                    strategies = pd.concat([strategies, pd.DataFrame({'Driver': [driver], 'Strategy': [[strategy_compounds, strategy_lengths]]})], ignore_index=True)

            qualifying_results = session.results[['Abbreviation', 'GridPosition']]
            race_results = session.results[['Abbreviation', 'Position']]

            # Calculate position changes
            position_changes = pd.merge(qualifying_results, race_results, on='Abbreviation')
            position_changes['PositionChange'] = position_changes['GridPosition'] - position_changes['Position']

            # Calculate score to rate strategies - Could be a ML task in of itself
            position_changes['Score'] = position_changes['PositionChange'] + 0.5 * position_changes['Position'].map(points).fillna(0)

            # For each unique strategy (compounds only), find the average score
            strategy_scores = pd.DataFrame(columns=['Strategy', 'AverageScore', 'AverageFinishPosition', 'Count'])
            for _, row in strategies.iterrows():
                driver = row['Driver']
                strategy = row['Strategy'][0]  # Get compounds only
                driver_score = position_changes[position_changes['Abbreviation'] == driver]['Score'].values[0]
                driver_finish_position = position_changes[position_changes['Abbreviation'] == driver]['Position'].values[0]
                # Check if strategy already exists
                existing_entry = strategy_scores[strategy_scores['Strategy'] == strategy]
                if not existing_entry.empty:
                    idx = existing_entry.index[0]
                    strategy_scores.at[idx, 'AverageScore'] = (strategy_scores.at[idx, 'AverageScore'] + driver_score) / 2
                    strategy_scores.at[idx, 'AverageFinishPosition'] = (strategy_scores.at[idx, 'AverageFinishPosition'] + driver_finish_position) / 2
                    strategy_scores.at[idx, 'Count'] += 1
                else:
                    strategy_scores = pd.concat([strategy_scores, pd.DataFrame({'Strategy': [strategy], 'AverageScore': [driver_score], 'AverageFinishPosition': [driver_finish_position], 'Count': [1]})], ignore_index=True)

            # Best strategy is the one with the highest average score, or lowest average finish position in case of ties
            strategy_scores = strategy_scores.sort_values(by=['AverageScore', 'AverageFinishPosition'], ascending=[False, True]).reset_index(drop=True)
            best_strategy = strategy_scores.iloc[0]

            gp_df.loc[len(gp_df)] = [
                event['EventName'] + f" {year}",
                track_info.get('laps', None),
                track_info.get('length_km', None),
                track_info.get('turn_count', None),
                track_info.get('drs_zones', None),
                track_info.get('pit_lane_length_m', None),
                track_info.get('degradation_level', None),
                track_info.get('street_circuit', None),
                avg_track_temp,
                avg_air_temp,
                avg_humidity,
                rain,
                safety_car_laps,
                best_strategy['Strategy']
            ]

        except Exception as e:
            print(f"Could not load session for {event['EventName']} {year}: {e}")
            continue

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 complet

Loaded session for Australian Grand Prix 2018


req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
core        WARNING 	Car telemetry data is unavailable!
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
core        WARNING 	Failed to determine `Session.t0_date`!
logger      WARNING 	Failed to load telemetry data!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '77', '44', '10', '20', '27', '14', '2', '9', '31', '55', '16', '8', '18', '35', '11', '28', '7', '33', '3']
C:\Users\idris\AppData\Local\Temp\ipykernel_4384\3044379049.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or al

Loaded session for Bahrain Grand Prix 2018


core        WARNING 	Fixed incorrect tyre stint information for driver '33'
core        WARNING 	Fixed incorrect tyre stint information for driver '27'
core        WARNING 	Fixed incorrect tyre stint information for driver '55'
core        WARNING 	Fixed incorrect tyre stint information for driver '31'
core        WARNING 	Fixed incorrect tyre stint information for driver '11'
core        WARNING 	Fixed incorrect tyre stint information for driver '35'
core        WARNING 	Fixed incorrect tyre stint information for driver '8'
core        WARNING 	Fixed incorrect tyre stint information for driver '10'
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 3 completed the race distance 00:00.025000 before the recor

...

Loaded session for Mexico City Grand Prix 2025


req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.010000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '12', '1', '63', '81', '87', '30', '6', '27', '10', '23', '31', '55', '14', '43', '18', '22', '44', '16', '5']
C:\Users\idris\AppData\Local\Temp\ipykernel_4384\3044379049.py:81: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  strategy_score

Loaded session for São Paulo Grand Prix 2025


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No 

Loaded session for Las Vegas Grand Prix 2025


In [32]:
gp_df.loc[gp_df.grand_prix == 'Sakhir Grand Prix 2020', ['laps', 'length_km', 'turn_count', 'drs_zones']] = [87, 3.543, 11, 2]

pd.set_option('display.max_rows', None)
gp_df

,grand_prix,laps,length_km,turn_count,drs_zones,pit_lane_length_m,degradation_level,street_circuit,avg_track_temp,avg_air_temp,avg_humidity,rainfall_mm,safety_car_laps,best_strategy
0,Australian Grand Prix 2018,58,5.303,14,2,280,Medium,True,36.324324,24.077477,30.915315,0.045045,7,"(ULTRASOFT, SOFT)"
1,Bahrain Grand Prix 2018,57,5.412,15,3,440,High,False,32.198058,27.982524,47.363107,0.000000,3,"(SUPERSOFT, SOFT)"
2,Chinese Grand Prix 2018,56,5.451,16,2,370,High,False,37.019643,19.446429,24.089286,0.017857,5,"(ULTRASOFT, SOFT)"
3,Azerbaijan Grand Prix 2018,51,6.003,20,2,316,Low,True,25.251754,16.661404,45.651754,0.000000,13,"(ULTRASOFT, SOFT, SUPERSOFT)"
4,Spanish Grand Prix 2018,66,4.655,14,2,367,High,False,32.339048,16.050476,52.286667,0.790476,8,"(MEDIUM, MEDIUM)"
5,Monaco Grand Prix 2018,78,3.337,19,1,350,Low,True,34.609735,26.640708,50.892920,0.123894,2,"(ULTRASOFT, HYPERSOFT)"
6,Canadian Grand Prix 2018,70,4.361,14,2,404,Low,True,36.949057,20.523585,27.825472,0.000000,4,"(ULTRASOFT, SUPERSOFT)"
7,French Grand Prix 2018,53,5.842,15,2,423,Low,False,42.033333,24.546078,49.049020,0.009804,7,"(SUPERSOFT, SOFT)"
8,Austrian Grand Prix 2018,71,4.318,10,3,359,Medium,False,45.879348,23.176087,37.141304,0.000000,2,"(SUPERSOFT, SOFT)"
9,British Grand Prix 2018,52,5.891,18,2,510,High,False,52.169388,27.431633,38.739796,0.000000,10,"(SOFT, MEDIUM, SOFT)"


In [33]:
gp_df.to_csv('gp_data.csv', index=False)

In [36]:
import fastf1
import pandas as pd

session = fastf1.get_session(2019, 'Monza', 'Race')
session.load()
laps = session.laps


# Add compound, stint, and pit info
laps = session.laps

# Compute per-stint averages
stint_data = (
    laps.groupby(["Driver", "Stint", "Compound"])
    .agg(stint_length=("LapNumber", "count"))
    .reset_index()
)

statuses = session.results['Status'].fillna('').astype(str)
dnf_mask = (statuses != 'Finished') & (~statuses.str.startswith('+'))
dnf_drivers = session.results[dnf_mask]['Abbreviation'].tolist()
strategies = pd.DataFrame(columns=['Driver', 'Strategy'])

# Strategy will be a 2D array of (Compound, Stint Length). So Soft for 10 laps, Medium for 20 laps would be [['Soft', 'Medium'], [10, 20]]
for driver in stint_data['Driver'].unique():
    if driver not in dnf_drivers:
        driver_stints = stint_data[stint_data['Driver'] == driver]
        strategy_compounds = tuple(driver_stints['Compound'].tolist())
        strategy_lengths = tuple(driver_stints['stint_length'].tolist())
        strategies = pd.concat([strategies, pd.DataFrame({'Driver': [driver], 'Strategy': [[strategy_compounds, strategy_lengths]]})], ignore_index=True)

qualifying_results = session.results[['Abbreviation', 'GridPosition']]
race_results = session.results[['Abbreviation', 'Position']]

# Calculate position changes
position_changes = pd.merge(qualifying_results, race_results, on='Abbreviation')
position_changes['PositionChange'] = position_changes['GridPosition'] - position_changes['Position']

# Calculate score to rate strategies - Could be a ML task in of itself
position_changes['Score'] = position_changes['PositionChange'] + 0.5 * position_changes['Position'].map(points).fillna(0)

# For each unique strategy (compounds only), find the average score
strategy_scores = pd.DataFrame(columns=['Strategy', 'AverageScore', 'AverageFinishPosition', 'Count'])
for _, row in strategies.iterrows():
    driver = row['Driver']
    strategy = row['Strategy'][0]  # Get compounds only
    driver_score = position_changes[position_changes['Abbreviation'] == driver]['Score'].values[0]
    driver_finish_position = position_changes[position_changes['Abbreviation'] == driver]['Position'].values[0]
    # Check if strategy already exists
    existing_entry = strategy_scores[strategy_scores['Strategy'] == strategy]
    if not existing_entry.empty:
        idx = existing_entry.index[0]
        strategy_scores.at[idx, 'AverageScore'] = (strategy_scores.at[idx, 'AverageScore'] + driver_score) / 2
        strategy_scores.at[idx, 'AverageFinishPosition'] = (strategy_scores.at[idx, 'AverageFinishPosition'] + driver_finish_position) / 2
        strategy_scores.at[idx, 'Count'] += 1
    else:
        strategy_scores = pd.concat([strategy_scores, pd.DataFrame({'Strategy': [strategy], 'AverageScore': [driver_score], 'AverageFinishPosition': [driver_finish_position], 'Count': [1]})], ignore_index=True)

# Best strategy is the one with the highest average score, or lowest average finish position in case of ties
strategy_scores = strategy_scores.sort_values(by=['AverageScore', 'AverageFinishPosition'], ascending=[False, True]).reset_index(drop=True)
best_strategy = strategy_scores.iloc[0]

# print(best_strategy['Strategy'])
# print(strategy_scores)
print(position_changes)


core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.092000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '77', '44', '3', '27', '23', '11', '33', '99', '4', '10', '1

   Abbreviation  GridPosition  Position  PositionChange  Score
0           LEC           1.0       1.0             0.0   12.5
1           BOT           3.0       2.0             1.0   10.0
2           HAM           2.0       3.0            -1.0    6.5
3           RIC           5.0       4.0             1.0    7.0
4           HUL           6.0       5.0             1.0    6.0
5           ALB           8.0       6.0             2.0    6.0
6           PER          18.0       7.0            11.0   14.0
7           VER          19.0       8.0            11.0   13.0
8           GIO          10.0       9.0             1.0    2.0
9           NOR          16.0      10.0             6.0    6.5
10          GAS          17.0      11.0             6.0    6.0
11          STR           9.0      12.0            -3.0   -3.0
12          VET           4.0      13.0            -9.0   -9.0
13          RUS          14.0      14.0             0.0    0.0
14          RAI           0.0      15.0           -15.0

C:\Users\idris\AppData\Local\Temp\ipykernel_4384\3235954777.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  strategy_scores = pd.concat([strategy_scores, pd.DataFrame({'Strategy': [strategy], 'AverageScore': [driver_score], 'AverageFinishPosition': [driver_finish_position], 'Count': [1]})], ignore_index=True)
